In [105]:
import pandas as pd
import numpy as np
import os
import cv2
import shutil
from tqdm import tqdm_notebook

In [96]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# VARIABLES

In [2]:
# ROOT OF THE PROJECT
project_path = os.path.join('../')

# PATH TO IMAGES
data_path = os.path.join(project_path, 'Data')
images = os.path.join(data_path, 'Images')

# PATH TO MODEL
model_path = os.path.join(project_path, 'faster_rcnn_inception_resnet')

# PATH TO TRAINING FOLDER
training_path = os.path.join(project_path, 'Training')

In [3]:
CREATE_TF_RECORDS = False
TRAIN_MODEL = False
EXPORT_GRAPH = False
INSTALL_LIBS = True
MOVE_GENERATETF_FILE = True


# CLONNING AND SETUP TESNORFLOW API

For using the code below, you need to do following steps: <br>
1. Clone tensorflow repository (`git clone https://github.com/tensorflow/models.git`) 
2. Install it: <br>
    a. `pip install tensorflow-object-detection-api` <br>
    b. `python models/research/setup.py build` <br>
    c. `python models/research/setup.py install` <br>
    d. `python /content/models/research/slim/setup.py build` <br>
    e. `python /content/models/research/slim/setup.py install`
3. Install <b>proto</b> files 
<b>Tensorflow==1.* required !!!</b>

P.S. <br>
    Good guide for Windows ([https://medium.com/@marklabinski/installing-tensorflow-object-detection-api-on-windows-10-7a4eb83e1e7b])

In [4]:
if INSTALL_LIBS:
    !pip install tensorflow-object-detection-api
    !python models/research/setup.py build
    !python models/research/setup.py install
    !python models/research/slim/setup.py build
    !python models/research/slim/setup.py install

  Using cached tensorflow_object_detection_api-0.1.1.tar.gz (577 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached scipy-1.4.1-cp37-cp37m-win_amd64.whl (30.9 MB)

ERROR: imgaug 0.4.0 requires imageio, which is not installed.
ERROR: imgaug 0.4.0 requires scikit-image>=0.14.2, which is not installed.
ERROR: catboost 0.20.1 requires graphviz, which is not installed.
ERROR: catboost 0.20.1 requires plotly, which is not installed.
ERROR: botocore 1.13.49 has requirement docutils<0.16,>=0.10, but you'll have docutils 0.16 which is incompatible.



  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844519 sha256=5cb0b551a75ef0e4d90d8f7ec5af8e9ceda3e92eb8897b34851f425232e1311b
  Stored in directory: c:\users\mikhail_krupin\appdata\local\pip\cache\wheels\71\7e\a2\461ab817fbaef68ec9cc60df16d3669d1285f032e4c98179bf
Successfully built tensorflow-object-detection-api
  Attempting uninstall: scipy
    Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1


running build
running install
running bdist_egg
running egg_info
writing object_detection.egg-info\PKG-INFO
writing dependency_links to object_detection.egg-info\dependency_links.txt
writing requirements to object_detection.egg-info\requires.txt
writing top-level names to object_detection.egg-info\top_level.txt
reading manifest file 'object_detection.egg-info\SOURCES.txt'
writing manifest file 'object_detection.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\EGG-INFO
copying object_detection.egg-info\PKG-INFO -> build\bdist.win-amd64\egg\EGG-INFO
copying object_detection.egg-info\SOURCES.txt -> build\bdist.win-amd64\egg\EGG-INFO
copying object_detection.egg-info\dependency_links.txt -> build\bdist.win-amd64\egg\EGG-INFO
copying object_detection.egg-info\requires.txt -> build\bdist.win-amd64\egg\EGG-INFO
copying object_detection.egg-info\top_level.txt -> build\bdist.win-a


zip_safe flag not set; analyzing archive contents...



cython 0.29.15 is already the active version in easy-install.pth
Installing cygdb-script.py script to C:\Users\Mikhail_Krupin\Anaconda\Scripts
Installing cygdb.exe script to C:\Users\Mikhail_Krupin\Anaconda\Scripts
Installing cython-script.py script to C:\Users\Mikhail_Krupin\Anaconda\Scripts
Installing cython.exe script to C:\Users\Mikhail_Krupin\Anaconda\Scripts
Installing cythonize-script.py script to C:\Users\Mikhail_Krupin\Anaconda\Scripts
Installing cythonize.exe script to C:\Users\Mikhail_Krupin\Anaconda\Scripts

Using c:\users\mikhail_krupin\anaconda\lib\site-packages\cython-0.29.15-py3.7-win-amd64.egg
Searching for matplotlib==3.2.1
Best match: matplotlib 3.2.1
Adding matplotlib 3.2.1 to easy-install.pth file

Using c:\users\mikhail_krupin\anaconda\lib\site-packages
Searching for pillow==7.0.0
Best match: pillow 7.0.0
Processing pillow-7.0.0-py3.7-win-amd64.egg
pillow 7.0.0 is already the active version in easy-install.pth

Using c:\users\mikhail_krupin\anaconda\lib\site-pack


zip_safe flag not set; analyzing archive contents...


copying slim.egg-info\PKG-INFO -> build\bdist.win-amd64\egg\EGG-INFO
copying slim.egg-info\SOURCES.txt -> build\bdist.win-amd64\egg\EGG-INFO
copying slim.egg-info\dependency_links.txt -> build\bdist.win-amd64\egg\EGG-INFO
copying slim.egg-info\top_level.txt -> build\bdist.win-amd64\egg\EGG-INFO
creating 'dist\slim-0.1-py3.7.egg' and adding 'build\bdist.win-amd64\egg' to it
removing 'build\bdist.win-amd64\egg' (and everything under it)
Processing slim-0.1-py3.7.egg
Removing c:\users\mikhail_krupin\anaconda\lib\site-packages\slim-0.1-py3.7.egg
Copying slim-0.1-py3.7.egg to c:\users\mikhail_krupin\anaconda\lib\site-packages
slim 0.1 is already the active version in easy-install.pth

Installed c:\users\mikhail_krupin\anaconda\lib\site-packages\slim-0.1-py3.7.egg
Processing dependencies for slim==0.1
Finished processing dependencies for slim==0.1


In [5]:
# MOVE generate_tfrecord.py file to models/research/object_detection folder

if MOVE_GENERATETF_FILE:
    source = 'generate_tfrecord.py'
    destination = 'models/research/object_detection/generate_tfrecord.py'

    shutil.copyfile(source, destination) 

# CREATING TF-RECORD FILES

In [0]:
# check that generate_tfrecord.py is in models/research/object_detection folder
# creating train and test .record files
"""
--csv_input ==> path to your annotation file
--output_path ==> path to folder, where your .record file will store
--image_dir ==> path to folder, where your images is stored 

"""
if CREATE_TF_RECORDS:
    !python models/research/object_detection/generate_tfrecord.py --csv_input='Data/train_annotation.csv' --output_path='Data/Records/train.record' --image_dir='Data/TrainImages'

data(filename='26423584250_612f177fef_b.jpg', object=                        filename        xmin  ...  width  height
0   26423584250_612f177fef_b.jpg  736.960157  ...  683.0  1024.0
1   26423584250_612f177fef_b.jpg  796.725031  ...  683.0  1024.0
2   26423584250_612f177fef_b.jpg  796.725031  ...  683.0  1024.0
3   26423584250_612f177fef_b.jpg  252.106890  ...  683.0  1024.0
4   26423584250_612f177fef_b.jpg  203.284834  ...  683.0  1024.0
5   26423584250_612f177fef_b.jpg  103.957267  ...  683.0  1024.0
6   26423584250_612f177fef_b.jpg  168.772734  ...  683.0  1024.0
7   26423584250_612f177fef_b.jpg  135.102343  ...  683.0  1024.0
8   26423584250_612f177fef_b.jpg   30.724222  ...  683.0  1024.0
9   26423584250_612f177fef_b.jpg    5.471447  ...  683.0  1024.0
10  26423584250_612f177fef_b.jpg   70.707768  ...  683.0  1024.0
11  26423584250_612f177fef_b.jpg   56.818742  ...  683.0  1024.0
12  26423584250_612f177fef_b.jpg  279.043157  ...  683.0  1024.0
13  26423584250_612f177fef_b.jpg  189



W0224 08:38:53.866143 140222552999808 module_wrapper.py:139] From models/research/object_detection/generate_tfrecord.py:90: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0224 08:38:55.370845 140222552999808 module_wrapper.py:139] From models/research/object_detection/generate_tfrecord.py:49: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



# TRAINING

In [0]:
"""
--train_dir ==> folder, where checkpoints and intermediate results of model will be stored
--pipeline_config_path ==> path to config file

"""
if TRAIN_MODEL:   
    !python models/research/object_detection/legacy/train.py --logstderr --train_dir='Training' --pipeline_config_path='faster_rcnn_inception_resnet/pipeline.config'

Process is terminated.


# EXPORT INFERENCE GRAPH

In [0]:
"""
--pipeline_config_path ==> path to train config file
--trained_checkpoint_prefix ==> path to checkpoint file. Replace {} to last checkpoint in folder. This checkpoint should have all 3 files!
--output_directory ==> path to folder, where to export inference graph
"""
if EXPORT_GRAPH:
    !python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path "Training/pipeline.config" --trained_checkpoint_prefix "Training/model.ckpt-{}" --output_directory "Training/Inference_graph" 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0226 10:20:10.331665 140621368240000 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0226 10:20:11.247793 140621368240000 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/object_detection/exporter.py:357: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0226 10:20:11.250188 140621368240000 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/object_detection/exporter.py:110: The name tf.placeholder is deprecated. Please use tf.

# TESTING

## DEPENDENCIES

In [4]:
import tensorflow as tf
%matplotlib inline

tf.__version__

'1.15.0'

In [5]:
import sys
sys.path.append("C:\\Users\\Mikhail_Krupin\\Jupyter\\Diploma\\models\\research\\")
sys.path.append("C:\\Users\\Mikhail_Krupin\\Jupyter\\Diploma\\models\\research\\object_detection\\utils")

In [6]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [7]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [8]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## VARIABLES

In [9]:
# Path to frozen detection graph. 
PATH_TO_FROZEN_GRAPH = 'Inference_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'Data/label_map.pbtxt'

#Number of classes
NUM_CLASSES = 3

## MODELING

In [10]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [17]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [12]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [13]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
              ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                     feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [110]:
# Path to folder with test images 
PATH_TO_TEST_IMAGES_DIR = 'Data/Test'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, i) for i in os.listdir(PATH_TO_TEST_IMAGES_DIR)]

# Size, in inches, of the output images.
IMAGE_SIZE = (19, 11)

In [114]:
def get_bbox_info(TEST_IMAGE_PATHS, isShow=False):
    c = 0

    outputs = []
    for image_path in TEST_IMAGE_PATHS:
        image = Image.open(image_path)
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        output_dict = run_inference_for_single_image(image_np, detection_graph)
        
        if isShow:
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              output_dict['detection_boxes'],
              output_dict['detection_classes'],
              output_dict['detection_scores'],
              category_index,
              instance_masks=output_dict.get('detection_masks'),
              use_normalized_coordinates=True,
              line_thickness=6)
            plt.figure(figsize=IMAGE_SIZE)
            plt.grid(False)
            plt.imshow(image_np)

            plt.savefig(f'img_pedestrian_{c}.jpg')
            c += 1

        output_dict['image_name'] = image_path
        outputs.append(output_dict)

    return outputs


In [ ]:
outputs = get_bbox_info(TEST_IMAGE_PATHS)

1. Pedestrians >>> 1
2. Non-pedestrians >>> 2
3. Road >>> 3

In [21]:
outputs[:2]

[{'num_detections': 9,
  'detection_boxes': array([[0.36005092, 0.04512974, 0.68201566, 0.1992135 ],
         [0.3227812 , 0.19779213, 0.7255821 , 0.32947472],
         [0.70621383, 0.        , 0.94982153, 1.        ],
         [0.35564336, 0.9415925 , 0.43585932, 0.9786588 ],
         [0.34804142, 0.3317642 , 0.7686305 , 0.506405  ],
         [0.3616256 , 0.8374388 , 0.4320923 , 0.87164867],
         [0.36344332, 0.72158563, 0.43094948, 0.7516091 ],
         [0.3463424 , 0.        , 0.68870574, 0.05666066],
         [0.66170543, 0.08714091, 0.89761066, 1.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.  

In [22]:
def preprocess_output_data(outputs):

    detections = []
    for output in outputs:

        current_detections = dict()
        number_of_detections = output['num_detections']

        detection_boxes = output['detection_boxes'][:number_of_detections]
        detection_classes = output['detection_classes'][:number_of_detections]
        detection_scores = output['detection_scores'][:number_of_detections]
        image_path = output['image_name']

        current_detections['boxes'] = detection_boxes
        current_detections['classes'] = detection_classes
        current_detections['scores'] = detection_scores
        current_detections['image_path'] = image_path
        current_detections['detected_objs'] = number_of_detections

        detections.append(current_detections)

    return detections

detections = preprocess_output_data(outputs)

In [23]:
detections[0]

{'boxes': array([[0.36005092, 0.04512974, 0.68201566, 0.1992135 ],
        [0.3227812 , 0.19779213, 0.7255821 , 0.32947472],
        [0.70621383, 0.        , 0.94982153, 1.        ],
        [0.35564336, 0.9415925 , 0.43585932, 0.9786588 ],
        [0.34804142, 0.3317642 , 0.7686305 , 0.506405  ],
        [0.3616256 , 0.8374388 , 0.4320923 , 0.87164867],
        [0.36344332, 0.72158563, 0.43094948, 0.7516091 ],
        [0.3463424 , 0.        , 0.68870574, 0.05666066],
        [0.66170543, 0.08714091, 0.89761066, 1.        ]], dtype=float32),
 'classes': array([1, 1, 3, 1, 1, 1, 2, 1, 3], dtype=uint8),
 'scores': array([0.99856806, 0.99854064, 0.9975932 , 0.9969047 , 0.9965803 ,
        0.9955498 , 0.98957753, 0.9541945 , 0.29939154], dtype=float32),
 'image_path': 'Data/Test\\image_pedestrian_0.jpg',
 'detected_objs': 9}

In [54]:
one_detection_dict = detections[0]

## METRICS

### Create ground truth files

In [136]:
metrics_folder = os.path.join('../Metrics')
ground_truth_folder = os.path.join(metrics_folder, 'ground-truth')
predicted_folder = os.path.join(metrics_folder, 'detection-results')

if not os.path.exists(metrics_folder):
    os.mkdir(metrics_folder)

if not os.path.exists(ground_truth_folder):
    os.mkdir(ground_truth_folder)
    
if not os.path.exists(predicted_folder):
    os.mkdir(predicted_folder)


In [134]:
class_names_dict = {1: 'Pedestrian', 2: 'Non-Pedestrian', 3: 'Road'}

def create_data_for_metrics(bbox_outputs):
    main_annotation = pd.read_csv(os.path.join(data_path, ('annotation.csv')))
    
    print('Creating Predicted Files...')
    # Creating predicted
    bbox_len = len(bbox_outputs)
    for  bbox_ind in tqdm_notebook(range(bbox_len), desc='PREDICTED FILES'):
        
        bbox = bbox_outputs[bbox_ind]
        
        length = bbox['detected_objs']
        img_name = bbox['image_path'].split('\\')[-1]
        
        height, width, _ = cv2.imread(bbox['image_path']).shape
        
        path = os.path.join(predicted_folder, img_name).replace('.jpg', '.txt')
        with open(path, 'w') as f_obj:
            
            for i in range(length):
                cls = bbox['classes'][i]
                cls = class_names_dict[cls] 
                
                score = bbox['scores'][i]
                
                ymin_left = bbox['boxes'][i][0] * height
                xmin_top = bbox['boxes'][i][1] * width
                ymax_right = bbox['boxes'][i][2] * height
                xmax_bottom = bbox['boxes'][i][3] * width
                
                row = f'{cls} {score} {ymin_left} {xmin_top} {ymax_right} {xmax_bottom}\n'
                
                f_obj.write(row)
            
    print('Creating Ground Truth Files...')
    # Creating ground truth
    for bbox_ind in tqdm_notebook(range(bbox_len), desc='PREDICTED FILES'):
        
        bbox = bbox_outputs[bbox_ind]
        
        length = bbox['detected_objs']
        img_name = bbox['image_path'].split('\\')[-1]
        
        path = os.path.join(ground_truth_folder, img_name).replace('.jpg', '.txt')
        with open(path, 'w') as f_obj:
                
            cur_img = main_annotation[main_annotation.filename == img_name]
            
            cur_img = cur_img[['class', 'ymin', 'xmin', 'ymax', 'xmax']]
            
            f_obj.write(cur_img.to_string(header=False, index=False))
            

In [115]:
print('Predicting Bounding Boxes...')
bbox_outputs = get_bbox_info(TEST_IMAGE_PATHS)
bbox_outputs = preprocess_output_data(bbox_outputs)

create_data_for_metrics(bbox_outputs)

Predicting Bounding Boxes...
Creating Predicted Files...


C:\Users\Mikhail_Krupin\Anaconda\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Creating Ground Truth Files...


C:\Users\Mikhail_Krupin\Anaconda\lib\site-packages\ipykernel_launcher.py:38: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


### Calculating metrics

In [141]:
!python mAP/main.py -na


C:\Users\Mikhail_Krupin\Jupyter\Diploma\metrics\ground-truth C:\Users\Mikhail_Krupin\Jupyter\Diploma\metrics\detection-results

OMP: Info #212: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #210: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: 0-7
OMP: Info #156: KMP_AFFINITY: 8 available OS procs
OMP: Info #157: KMP_AFFINITY: Uniform topology
OMP: Info #179: KMP_AFFINITY: 1 packages x 4 cores/pkg x 2 threads/core (4 total cores)
OMP: Info #214: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 core 0 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 core 0 thread 1 



83.27% = Non-Pedestrian AP 
99.11% = Pedestrian AP 
95.44% = Road AP 
mAP = 92.61%


OMP: Info #171: KMP_AFFINITY: OS proc 2 maps to package 0 core 1 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 3 maps to package 0 core 1 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 4 maps to package 0 core 2 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 5 maps to package 0 core 2 thread 1 
OMP: Info #171: KMP_AFFINITY: OS proc 6 maps to package 0 core 3 thread 0 
OMP: Info #171: KMP_AFFINITY: OS proc 7 maps to package 0 core 3 thread 1 
OMP: Info #250: KMP_AFFINITY: pid 28104 tid 3672 thread 0 bound to OS proc set 0
Traceback (most recent call last):
  File "mAP/main.py", line 754, in <module>
cv2.error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'



# CALCULATING OF OPTIMAL  GREEN INTERVAL

FORMULA: <br>
\begin{align}
T_{opt} = t_1 + \frac{B}{V_p} + log_{10}(n+1)*n + w_b * [b == 1] + w_{ch} * [ch == 1] + w_{old} * [old == 1] - w_r * [r == 1] 
\end{align}

$t_1$ is the starting delay of the first row of pedestrians once a green traffic light turns on (`const 3 sec`) <br>
$B$ is the width of the traffic way in meters <br>
$V_p$ is the speed of pedestrians (varies greatly from `0.5 m/s` to `1.5 m/s` <br>
$n$ - number of pedestrians, which will cross the road <br>
<b>$[b == 1]$</b> - есть ли рядом места большого скопления народа
$w_b$ - вес для $[b == 1]$ <br>
$[ch == 1]$ - текущий момент это период времени, когда много мам с колясками <br>
$w_{ch}$ - вес для события $[ch == 1]$ <br>
$[old == 1]$ - текущий момент это период времени, когда много пожилых людей <br>
$w_{old}$ - вес для $[old == 1]$ <br>
$[r == 1]$ - текущий момент это период времени, когда сильный поток машин на дороге <br>
$w_{r}$ - вес для $[r == 1]$ <br>

In [26]:
# constant parameters
t1 = 3
V_p = 1.2

# user parameters
N = 1
road_width = 18

# coefs for our events
w_b = np.random.normal(6, 2, 1) 
w_ch = np.random.normal(7, 2, 1)
w_old = np.random.normal(8, 2, 1)
w_r = np.random.normal(6, 2, 1)

# events
b = 0
ch = 0
old = 0
r = 0

Description: <br>
1. <b>Constant parameters</b> - this one will not change <br>
2. <b>User parameters</b> - this one will enter user (the width of a road) and will be received from object detection algorithm (number of pedestrians)
3. <b>Coefs for events</b> - this parameters should be got from a client 
4. <b>Events</b> - we will take pictures from video cameras and then will get time from them. After this we will get info, which events are happening  

In [27]:
params = {
    'N': N,
    'w_b': w_b,
    'w_ch': w_ch,
    'w_old': w_old,
    'w_r': w_r,
    'V_p': V_p,
    'isCrowdy': b,
    'isManyChildren': ch,
    'isManyElders': old,
    'isManyCars': r,
    't1': t1,
    'B': road_width
}

def calculate_optimal_time(params):
    
    # calculating green interval in seconds
    result = params['t1'] + params['B'] / params['V_p'] + np.log(params['N'] + 1) * params['N'] + \
                params['w_b'] * params['isCrowdy'] + params['w_ch'] * params['isManyChildren'] + \
                params['w_old'] * params['isManyElders'] + params['w_r'] * params['isManyCars']
    
    return result
    
    

In [28]:
calculate_optimal_time(params)

array([18.69314718])

# FULL PIPELINE

In [45]:
def gather_params_for_calculating(bbox_outputs, client_coefs, road_width):
    
    print(bbox_outputs['classes'])
    num_pedestrians = np.where(bbox_outputs['classes'] == 1)[0].shape[0]
    
    # Constant parameters
    t1 = 3
    V_p = 1.2
    
    # Events (will calculate after taking the picuters from video camera)
    b = 0
    ch = 0
    old = 0
    r = 0
    
    params = {
        'N': num_pedestrians,
        'w_b': client_coefs['w_b'],
        'w_ch': client_coefs['w_ch'],
        'w_old': client_coefs['w_old'],
        'w_r': client_coefs['w_r'],
        'V_p': V_p,
        'isCrowdy': b,
        'isManyChildren': ch,
        'isManyElders': old,
        'isManyCars': r,
        't1': t1,
        'B': road_width
        
    }
    
    return params

In [50]:
def get_optimal_time(TEST_IMAGE_PATHS, client_coefs, road_width):
    
    bbox_output_info = get_bbox_info(TEST_IMAGE_PATHS)
    preprocessed_box_info = preprocess_output_data(bbox_output_info)
    
    optimal_times = dict()
    for image in preprocessed_box_info:
        img_path = image['image_path']

        params = gather_params_for_calculating(image, client_coefs, road_width)
        optimal_time = calculate_optimal_time(params)
        
        optimal_times[img_path] = optimal_time
    
    return optimal_times

In [51]:
# TEST DATA
# Test client's coefs
w_b = np.random.normal(6, 2, 1) 
w_ch = np.random.normal(7, 2, 1)
w_old = np.random.normal(8, 2, 1)
w_r = np.random.normal(6, 2, 1)

client_coefs = {
    'w_b': w_b,
    'w_ch': w_ch,
    'w_old': w_old,
    'w_r': w_r
}

# Test road_width
road_width = 18

# CALCULATING OPTIMAL TIME
opt_time = get_optimal_time(TEST_IMAGE_PATHS[:1], client_coefs, road_width)

[1 1 3 1 1 1 2 1 3]


In [52]:
opt_time

{'Data/Test\\image_pedestrian_0.jpg': array([29.67546089])}